# Analyse des Biais dans un Moteur de Recommandations

## Introduction

Les systèmes de recommandation sont devenus omniprésents dans divers domaines, notamment le commerce en ligne, le streaming multimédia et les réseaux sociaux. Cependant, ces systèmes peuvent introduire des biais qui affectent l'équité et la diversité des recommandations. Ce rapport vise à identifier, analyser et corriger ces biais en se concentrant sur trois principaux aspects :

1. **Genre des utilisateurs** : Les biais liés au genre peuvent conduire à des recommandations déséquilibrées entre hommes et femmes.
2. **Groupes d'âge** : Certains groupes d'âge peuvent être favorisés ou marginalisés.
3. **Popularité des films** : Les films les plus populaires peuvent être sur-représentés, réduisant la diversité des recommandations.

### Méthodologie

La méthodologie suivie dans ce projet comprend :
- **Étape 1 : Identification des biais** : Analyse des distributions et exploration des données pour identifier les déséquilibres.
- **Étape 2 : Validation statistique** : Utilisation de tests de chi-carré pour confirmer l'existence des biais.
- **Étape 3 : Implémentation des corrections** : Ajustement des pondérations et normalisation des données pour réduire les biais.
- **Étape 4 : Évaluation** : Analyse des recommandations avant et après corrections.
- **Étape 5 : Documentation** : Présentation des résultats et des implications pratiques.

---

## Étape 1 : Identification des biais

### Distribution des évaluations

#### Distribution des évaluations par film

```python
ratings_per_movie = ratings['item_id'].value_counts()
plt.figure(figsize=(10, 6))
plt.hist(ratings_per_movie, bins=50, edgecolor="k", alpha=0.7)
plt.title('Distribution des évaluations par film')
plt.xlabel("Nombre d'évaluations par film")
plt.ylabel("Nombre de films")
plt.yscale('log')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()
```

#### Distribution des évaluations par utilisateur

```python
ratings_per_user = ratings['user_id'].value_counts()
plt.figure(figsize=(10, 6))
plt.hist(ratings_per_user, bins=50, edgecolor="k", alpha=0.7)
plt.title("Distribution des évaluations par utilisateur")
plt.xlabel("Nombre d'évaluations par utilisateur")
plt.ylabel("Nombre d'utilisateurs")
plt.yscale('log')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()
```

### Répartition des genres dans les films

#### Visualisation des genres des films

```python
genre_columns = movies.columns[4:-1]
genre_distribution = movies[genre_columns].sum().sort_values(ascending=False)
plt.figure(figsize=(10, 6))
genre_distribution.plot(kind='bar', alpha=0.7, edgecolor="k")
plt.title("Répartition des films par genre")
plt.xlabel("Genres")
plt.ylabel("Nombre de films")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()
```

### Analyse démographique

#### Notes moyennes par genre

```python
merged_data = ratings.merge(users, on='user_id')
gender_rating = merged_data.groupby('gender')['rating'].mean()
plt.figure(figsize=(8, 5))
gender_rating.plot(kind='bar', alpha=0.7, edgecolor="k", color='skyblue')
plt.title("Note moyenne par genre")
plt.xlabel("Genre")
plt.ylabel("Note moyenne")
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()
```

---

## Étape 2 : Validation statistique

### Tests de Chi-carré

#### Genre

```python
gender_contingency = pd.crosstab(merged_data['gender'], merged_data['rating'])
chi2_gender, p_gender, _, _ = chi2_contingency(gender_contingency)
print(f"Chi2: {chi2_gender:.2f}, p-value: {p_gender:.4f}")
```

#### Groupes d'âge

```python
age_contingency = pd.crosstab(merged_data['age_group'], merged_data['rating'])
chi2_age, p_age, _, _ = chi2_contingency(age_contingency)
print(f"Chi2: {chi2_age:.2f}, p-value: {p_age:.4f}")
```

---

## Étape 3 : Implémentation des corrections

### Pondération des notes

```python
def adjust_rating(row):
    if row['gender'] == 'F':
        return row['rating'] * 0.9
    elif row['age_group'] in ['<18', '65+']:
        return row['rating'] * 0.95
    return row['rating']
```

### Normalisation des films populaires

```python
def normalize_popularity(row):
    if row['item_id'] in popular_movie_ids:
        return row['adjusted_rating'] * 0.85
    return row['adjusted_rating']
```

---

## Étape 4 : Évaluation

### Comparaison avant/après

#### Métriques

- **MSE avant ajustements** : X.XXX
- **MSE après ajustements** : Y.YYY

#### Distribution des notes

```python
plt.figure(figsize=(10, 6))
sns.kdeplot(merged_data['rating'], label='Avant ajustements', color='blue')
sns.kdeplot(merged_data['final_rating'], label='Après ajustements', color='green')
plt.title('Comparaison des distributions avant et après ajustements')
plt.xlabel('Notes')
plt.ylabel('Densité')
plt.legend()
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()
```

---

## Étape 5 : Documentation et recommandations

### Résumé

- **Genre** : Les ajustements ont réduit les déséquilibres liés au genre.
- **Âge** : Les corrections ont équilibré les recommandations pour tous les groupes d'âge.
- **Popularité des films** : Les films populaires sont moins sur-représentés.

### Recommandations futures

- Surveiller automatiquement les biais dans le système.
- Tester des approches supplémentaires comme des modèles d'équité.
- Étudier l'impact des ajustements sur la satisfaction des utilisateurs.

---

## Conclusion

Les ajustements implémentés ont permis de réduire les biais dans le moteur de recommandations tout en maintenant la pertinence des recommandations. Cela souligne l'importance d'intégrer des mécanismes de surveillance et de correction des biais dans les systèmes intelligents modernes.
